# LastMile Instrumentor for LlamaIndex

In this notebook, we showcase how to use the **LastMile Tracing SDK** to auto-instrument tracing for your LlamaIndex applications. With tracing automatically setup, you can easily debug your RAG application using LastMile's RAG Debugger.

## Notebook Outline
* [Step 1: Setup](#setup)
* [Step 2: Configure the LastMile Instrumentor](#step2)
* [Step 3: Load and Process Docs with LlamaIndex](#step3)
* [Step 4: Create an Index and Query Engine with LlamaIndex](#step4)
* [Step 5: Query the Index with LlamaIndex](#step5)
* [Step 6: View Trace Data in RAG Debugger](#step6)


<a name="setup"></a>
# Step 1: Setup

To begin, we need to install a few packages including llamaindex and lastmile-eval.


In [ ]:
!pip install llama-index-embeddings-openai --upgrade
%pip install -q html2text llama-index pandas pyarrow tqdm
%pip install -q llama-index-readers-web
%pip install -q llama-index-callbacks-openinference
!pip install openai --upgrade
!pip install "tracing-auto-instrumentation[llama-index]" --upgrade


Import the necessary libraries for this example

In [ ]:
import llama_index.core

from tracing_auto_instrumentation.llama_index import LlamaIndexCallbackHandler
import textwrap

Before we start this tutorial, we need the following tokens/keys:

* LastMile AI API Token: Go to the [LastMile Settings page](https://lastmileai.dev/settings?page=tokens). You will need to first create a LastMile AI account.
* OpenAI API Key: Go to [OpenAI API Keys page](https://platform.openai.com/account/api-keys) to create and access your OpenAI API Key.

You can either set these explicitly (uncomment the lines below), or save them in a `.env` file within this project directory.

In [ ]:

import dotenv
dotenv.load_dotenv()

# import os
# os.environ['OPENAI_API_KEY'] =  os.getenv('OPENAI_API_KEY')
# os.environ['LASTMILE_API_TOKEN'] =  os.getenv('LASTMILE_API_TOKEN')

<a name="step2"></a>

## Step 2: Configure the LastMile Instrumentor

Next, we need to configure the LastMile Instrumentor by setting the global handler for LlamaIndex.

In [2]:
import llama_index.core

from tracing_auto_instrumentation.llama_index import LlamaIndexCallbackHandler

# llama_index.core.global_handler = LlamaIndexCallbackHandler(
#     project_name="LlamaIndex with Paul Graham",
# )

print(LlamaIndexCallbackHandler(
    project_name="LlamaIndex with Paul Graham",
)._tracer.project_id)


[WARNING] 2024-06-10 17:07:01,485 __init__.py:518: Overriding of current TracerProvider is not allowed
[DEBUG] 2024-06-10 17:07:01,487 connectionpool.py:1052: Starting new HTTPS connection (1): lastmileai.dev:443
[DEBUG] 2024-06-10 17:07:02,024 connectionpool.py:546: https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=LlamaIndex+with+Paul+Graham HTTP/1.1" 200 362


clx9n70lh00e7qj0qk5njyotx


<a name="step3"></a>

# Step 3: Load and Process Documents


In [3]:
!rag-debug launch


/Users/ankush/anaconda3/envs/empty/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_parsers" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/ankush/anaconda3/envs/empty/lib/python3.12/site-packages/pydantic/_internal/_fields.py:160: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
[INFO] 2024-06-10 17:07:24,066 cli.py:45: Running subcommand
INFO:     Started server process [66932]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     127.0.0.1:55725 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:55729 - "GET /manifest.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:55730 - "GET /f

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader().load_data(
    [
        "https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt"
    ]
)

parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

<a name="step4"></a>

# Step 4: Create an Index and Query Engine

In [ ]:
from llama_index.embeddings.openai.base import OpenAIEmbedding

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()


<a name="step5"></a>

# Step 5: Query the Index

In [ ]:
max_characters_per_line = 80
queries = [
    "What did Paul Graham do growing up?",
    "When and how did Paul Graham's mother die?",
    "What, in Paul Graham's opinion, is the most distinctive thing about YC?"
]
for query in queries:
    response = query_engine.query(query)
    print("Query")
    print("=====")
    print(textwrap.fill(query, max_characters_per_line))
    print()
    print("Response")
    print("========")
    print(textwrap.fill(str(response), max_characters_per_line))
    print()

<a name="step6"></a>

# Step 6: View Trace Data in RAG Debugger
Now we can view the trace data of our LlamaIndex application in a UI!
#### From your terminal:

Export your LASTMILE_API_TOKEN

```bash
export LASTMILE_API_TOKEN="<your-api-token>"
```

Run this CLI command to access the UI

```bash
rag-debug launch
```
Navigate to the 'Traces' Page where you see all the Traces listed under this Project "LlamaIndex with Paul Graham" (top-right corner to choose Project).

<img width="973" alt="Screenshot 2024-05-28 at 11 37 26 AM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/9568199a-5404-4254-aaf5-f87ac5a2f562"/>

Let's click into the Trace.

<img width="973" alt="Screenshot 2024-05-28 at 11 38 05 AM" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/cc1e451c-472f-4b56-9bfc-5508326d12d9"/>

Here we can see all the spans auto-generated for us. This can help us debug and pinpoint issues in our application, especially if we add additional logging on top of the auto-instrumentor.